# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# enter your code here
orders = pd.read_csv("../../lab-subsetting-and-descriptive-stats/your-code/Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
#Aggregate amount spent for unique customers

orders_aggregate = orders.groupby('CustomerID').agg('sum')['amount_spent'].reset_index()
orders_aggregate.head()


,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


In [4]:
#Select customers chose amount spent is in iQR

q75 = orders_aggregate['amount_spent'].quantile(0.75)
q95 = orders_aggregate['amount_spent'].quantile(0.95)

orders_aggregate.loc[(orders_aggregate['amount_spent'] >=q75) & (orders_aggregate['amount_spent'] <= q95)].head()


,CustomerID,amount_spent
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
5,12352,2506.04
9,12356,2811.43


In [5]:
#label selected customers as "VIP" or "Preferred"
#VIP Customers above the 95th percentile. Preferred Customers between the 75th and 95th percentile.

def Label(amount):
    if q75 < amount < q95:
        return 'Preferred Customers'
    elif amount > q95:
        return 'VIP Customers'
    else:
        return 'Other'

orders_aggregate['Label'] = orders_aggregate['amount_spent'].apply(Label)
orders_aggregate.head()

,CustomerID,amount_spent,Label
0,12346,77183.60,VIP Customers
1,12347,4310.00,Preferred Customers
2,12348,1797.24,Preferred Customers
3,12349,1757.55,Preferred Customers
4,12350,334.40,Other


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [6]:
# Country with most VIP Customers.
# Repito la misma operación que al princpio del challenge pero agrupando por países
orders_country = orders.groupby(['Country']).agg('sum')['amount_spent'].reset_index()
orders_country['Label'] = orders_country['amount_spent'].apply(Label)
orders_country.head()

,Country,amount_spent,Label
0,Australia,138521.31,VIP Customers
1,Austria,10198.68,VIP Customers
2,Bahrain,548.40,Other
3,Belgium,41196.34,VIP Customers
4,Brazil,1143.60,Other


In [15]:
# Agrupa ahora por país y etiqueta para ver cuál suma + cantidad de dinero y, por tanto, más VIP Customers.
orders_country = orders_country.groupby(['Country', 'Label']).agg('sum')['amount_spent'].reset_index().sort_values(by='amount_spent', ascending=False)
orders_country.head()

#United Kingdom tiene más VIP Customers


,Country,Label,amount_spent
35,United Kingdom,VIP Customers,7.308392e+06
23,Netherlands,VIP Customers,2.854463e+05
10,EIRE,VIP Customers,2.655459e+05
14,Germany,VIP Customers,2.288671e+05
13,France,VIP Customers,2.090240e+05


In [16]:
#Q3 : Country with VIP and Customers Preferred combined
